In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
data = pd.read_csv('train_titanic.csv')

In [3]:
data.drop('Name',axis=1,inplace=True)

In [4]:
data.drop('Ticket',axis=1,inplace=True)
data.drop('Fare',axis=1,inplace=True)

In [5]:
data.drop('Cabin',axis=1,inplace=True)

In [6]:
def gen(s):
    if s=="male":
        return 1
    else:
        return 0
data["Gender"]=data["Sex"].apply(gen)

In [7]:
del data["Sex"]

In [8]:
meanM= data[data.Gender==1].Age.mean()

In [9]:
data["age"]=np.where(pd.isnull(data.Age) & data["Gender"]==1  ,meanM, data["Age"])

In [10]:
meanF= data[data.Gender==0].Age.mean()

In [11]:
data.age.fillna(meanF,inplace=True)

In [12]:
del data["Age"]

In [13]:
data.dropna(inplace=True)

In [14]:
def emb(s):
    if s=="S":
        return 1
    elif s=="C":
        return 2
    else:
        return 3
data["Emb"]=data["Embarked"].apply(emb)
del data["Embarked"]

In [15]:
data.rename(columns={'age':'Age'},inplace=True)
data.rename(columns={'Gender':'Sex'},inplace=True)

In [16]:
data.rename(columns={'Emb':'Embarked'},inplace=True)

In [17]:
data = data[['Pclass','Sex','Age','SibSp','Parch','Embarked','Survived']]

In [18]:
test=pd.read_csv("test_titanic.csv")

In [19]:
del test["Name"]
del test["Ticket"]
del test["Fare"]
del test["Cabin"]

In [20]:
test["Emb"]=test["Embarked"].apply(emb)

In [21]:
test["Gender"]=test["Sex"].apply(gen)

In [22]:
del test["Sex"]
del test["Embarked"]

In [23]:
test.rename(columns={'Emb':'Embarked'},inplace=True)
test.rename(columns={'Gender':'Sex'},inplace=True)

In [24]:
test = test[['Pclass','Sex','Age','SibSp','Parch','Embarked']]

In [25]:
data=data.values

In [26]:
meanMT=test[test.Sex==1].Age.mean()

In [27]:
meanFT=test[test.Sex==0].Age.mean()

In [28]:
test["age"]=np.where(pd.isnull(test.Age) & test["Sex"]==1,meanMT,test["Age"])

In [29]:
test.age.fillna(meanFT,inplace=True)

In [30]:
del test["Age"]

In [31]:
test.rename(columns={'age':'Age'},inplace=True)

In [32]:
test=test[['Pclass','Sex','Age','SibSp','Parch','Embarked']]

In [33]:
test=test.values

In [34]:
def cost(data,m,h_x):
    M=data.shape[0]
    N=data.shape[1]
    total_cost=0
    for i in range(M):
        x=data[i,0:N-1]
        x=np.append(x,1)
        y=data[i,N-1]
        temp= (-y*np.log(h_x[i]))-((1-y)*np.log(1-h_x[i]))
        total_cost+=temp
    return total_cost/M

In [35]:
def predict(h_x):
    ypred=np.zeros(len(h_x))
    ypred=h_x>0.5
    ypred=ypred.astype(float)
    return ypred

In [36]:
def findh_x(m,data):
    M=data.shape[0]
    N=data.shape[1]
    h_x=np.zeros(M)
    for i in range(M):
        x=data[i,0:N-1]
        x=np.append(x,1)
        temp=(m.T*x).sum()
        h_x[i]=1/(1+np.exp(-temp))
    return h_x

In [37]:
def findm(data,learning_rate,m,h_x):
    M=data.shape[0]
    N=data.shape[1]
    new_m=np.zeros(N)
    slope_m=np.zeros(N)
    der_m=np.zeros(N)
    for i in range(M):
        x=data[i,0:N-1]
        x=np.append(x,1)
        y=data[i,N-1]
        for j in range(len(x)):
            slope_m[j]= (-x[j])*(y-h_x[i])
            der_m[j]+=slope_m[j]
    der_m=der_m/M
    new_m=m-learning_rate*der_m    
    return new_m

In [38]:
def logistic(data,learning_rate,num_iterations):
    m=np.zeros(data.shape[1])
    for i in range(num_iterations):
        h_x=findh_x(m,data)
        m=findm(data,learning_rate,m,h_x)
        #print("Cost: ",cost(data,m,h_x))
    return m

In [39]:
def test_data(m,data):
    M=data.shape[0]
    N=data.shape[1]
    h_x=np.zeros(M)
    for i in range(M):
        x=data[i,0:N]
        x=np.append(x,1)
        temp=(m.T*x).sum()
        h_x[i]=1/(1+np.exp(-temp))
    ypred=np.zeros(len(h_x))
    ypred=h_x>0.5
    ypred=ypred.astype(float)
    return ypred

In [40]:
def run():
    learning_rate=0.008
    num_iterations=1000
    m=logistic(data,learning_rate,num_iterations)
    print(m)
    h_x=findh_x(m,data)
    
    #print(h_x)
    #ypred=predict(h_x)
    count =0
    #print(ypred)
    #for i in range(data.shape[0]):
     #   if(ypred[i]==data[i,6]):
      #      count+=1
    #print(count/data.shape[0])
    ypred=test_data(m,test)
    print(ypred)

In [41]:
run()

[-0.36281653 -0.66927009  0.00098625 -0.03547578  0.13654337  0.35633235
  0.17612053]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 1. 0. 0.]
